In [ ]:
import subprocess
import sys
from pathlib import Path

BASE_DIR = Path('/home/wencm/Alimentación')
REQ_PATH = BASE_DIR / 'requirements.txt'
LIB_DIR = BASE_DIR / 'lib'
LIB_DIR.mkdir(exist_ok=True)

subprocess.check_call([
    sys.executable,
    '-m',
    'pip',
    'install',
    '--upgrade',
    '--target',
    str(LIB_DIR),
    '-r',
    str(REQ_PATH),
])

print(f'Requirements instalados en: {LIB_DIR}')


# Mercadona RAG Notebook

Este notebook hace RAG sobre `mercadona_data.xlsx`, recupera filas relevantes y llama a OpenAI para responder usando solo ese contexto.

## Requisitos
- Tener `OPENAI_API_KEY` en variables de entorno.
- Tener `openai`, `pandas`, `numpy`, `openpyxl` instalados en `/home/wencm/Alimentación/lib`.


In [1]:
import os
import sys
from pathlib import Path

LIB_DIR = Path('/home/wencm/Alimentación/lib')
if str(LIB_DIR) not in sys.path:
    sys.path.insert(0, str(LIB_DIR))

import numpy as np
import pandas as pd
from openai import OpenAI

BASE_DIR = Path('/home/wencm/Alimentación')
EXCEL_PATH = BASE_DIR / 'mercadona_data.xlsx'
CACHE_DIR = BASE_DIR / 'rag_cache'
CACHE_DIR.mkdir(exist_ok=True)
ENV_PATH = BASE_DIR / '.env'

CHUNKS_CSV = CACHE_DIR / 'chunks.csv'
EMB_NPY = CACHE_DIR / 'embeddings.npy'

EMBED_MODEL = 'text-embedding-3-small'
CHAT_MODEL = 'gpt-4.1-mini'

def load_env_file(path: Path) -> bool:
    if not path.exists():
        return False

    loaded_any = False
    for raw_line in path.read_text(encoding='utf-8').splitlines():
        line = raw_line.strip()
        if not line or line.startswith('#') or '=' not in line:
            continue

        key, value = line.split('=', 1)
        key = key.strip()
        value = value.strip()

        if (value.startswith('"') and value.endswith('"')) or (value.startswith("'") and value.endswith("'")):
            value = value[1:-1]

        if key and key not in os.environ:
            os.environ[key] = value
            loaded_any = True

    return loaded_any

if not os.getenv('OPENAI_API_KEY'):
    loaded = load_env_file(ENV_PATH)
    if loaded:
        print(f'Variables cargadas desde: {ENV_PATH}')

print(f'Excel: {EXCEL_PATH}')
print(f'Cache: {CACHE_DIR}')
print(f'Python: {sys.executable}')
print('OPENAI_API_KEY presente:', bool(os.getenv('OPENAI_API_KEY')))



Excel: /home/wencm/Alimentación/mercadona_data.xlsx
Cache: /home/wencm/Alimentación/rag_cache
Python: /home/wencm/WhaRAGBot/.venv/bin/python
OPENAI_API_KEY presente: True


In [2]:
def _clean(v):
    if pd.isna(v):
        return ''
    s = str(v).strip()
    return '' if s.lower() == 'nan' else s

def _numeric(v):
    if pd.isna(v):
        return ''
    try:
        f = float(v)
    except Exception:
        return _clean(v)
    return f'{f:g}'

def build_row_text(row):
    parts = []
    parts.append(f"Producto: {_clean(row.get('product_name'))}")
    parts.append(f"ID producto: {_numeric(row.get('product_id'))}")
    parts.append(f"Categoria: {_clean(row.get('category'))} > {_clean(row.get('subcategory'))} > {_clean(row.get('subsubcategory'))}")
    parts.append(f"Formato: {_clean(row.get('packaging'))}, {_numeric(row.get('unit_size'))} {_clean(row.get('size_format'))}")
    parts.append(f"Precio unidad: {_numeric(row.get('price_unit'))} EUR")
    parts.append(f"Precio por volumen/peso: {_numeric(row.get('price_bulk'))} EUR")
    parts.append(f"Ingredientes: {_clean(row.get('Ingredientes'))}")
    parts.append(f"Alergenos: {_clean(row.get('Alérgenos'))}")
    parts.append(
        'Nutricion por 100g/ml: '
        f"kJ={_numeric(row.get('nutrition_kj_100'))}; "
        f"kcal={_numeric(row.get('nutrition_kcal_100'))}; "
        f"grasas={_numeric(row.get('nutrition_fat_g_100'))} g; "
        f"saturadas={_numeric(row.get('nutrition_saturates_g_100'))} g; "
        f"hidratos={_numeric(row.get('nutrition_carbs_g_100'))} g; "
        f"azucares={_numeric(row.get('nutrition_sugars_g_100'))} g; "
        f"proteinas={_numeric(row.get('nutrition_protein_g_100'))} g; "
        f"sal={_numeric(row.get('nutrition_salt_g_100'))} g"
    )
    parts.append(f"Imagen principal: {_clean(row.get('thumbnail_url'))}")
    parts.append(f"Imagenes: {_clean(row.get('photo_urls'))}")
    return '\n'.join(parts)

df = pd.read_excel(EXCEL_PATH)
print('Filas:', len(df))

chunks = pd.DataFrame({
    'row_idx': np.arange(len(df), dtype=int),
    'product_id': df.get('product_id'),
    'product_name': df.get('product_name'),
    'category': df.get('category'),
    'subcategory': df.get('subcategory'),
    'subsubcategory': df.get('subsubcategory'),
    'packaging': df.get('packaging'),
    'unit_size': df.get('unit_size'),
    'size_format': df.get('size_format'),
    'price_unit': df.get('price_unit'),
    'price_bulk': df.get('price_bulk'),
    'ingredientes': df.get('Ingredientes'),
    'alergenos': df.get('Alérgenos'),
    'nutrition_ocr_text': df.get('nutrition_ocr_text'),
    'text': [build_row_text(r) for _, r in df.iterrows()],
})

chunks['lexical_text'] = (
    chunks['product_name'].fillna('').astype(str) + ' ' +
    chunks['category'].fillna('').astype(str) + ' ' +
    chunks['subcategory'].fillna('').astype(str) + ' ' +
    chunks['subsubcategory'].fillna('').astype(str) + ' ' +
    chunks['ingredientes'].fillna('').astype(str)
)

# Ingredient-focused name/index fields (high precision)
chunks['ingredient_search_text'] = (
    chunks['product_name'].fillna('').astype(str) + ' ' +
    chunks['category'].fillna('').astype(str) + ' ' +
    chunks['subcategory'].fillna('').astype(str) + ' ' +
    chunks['subsubcategory'].fillna('').astype(str)
)

# Ingredient-focused description fields (high recall)
chunks['ingredient_desc_text'] = (
    chunks['ingredientes'].fillna('').astype(str) + ' ' +
    chunks['alergenos'].fillna('').astype(str) + ' ' +
    chunks['nutrition_ocr_text'].fillna('').astype(str).str.slice(0, 2500)
)

chunks.to_csv(CHUNKS_CSV, index=False)
print('Chunks guardados en:', CHUNKS_CSV)
chunks.head(2)


Filas: 4553
Chunks guardados en: /home/wencm/Alimentación/rag_cache/chunks.csv


,row_idx,product_id,product_name,category,subcategory,subsubcategory,packaging,unit_size,size_format,price_unit,price_bulk,ingredientes,alergenos,nutrition_ocr_text,text,lexical_text,ingredient_search_text,ingredient_desc_text
0,0,4241.0,"Aceite de oliva 0,4º Hacendado","Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva,Garrafa,5.0,l,19.75,3.95,Ingredientes: Aceite de oliva refinado y Aceit...,x99.,Aceitede Oliva | Que contiene exclusivamente |...,"Producto: Aceite de oliva 0,4º Hacendado\nID p...","Aceite de oliva 0,4º Hacendado Aceite, especia...","Aceite de oliva 0,4º Hacendado Aceite, especia...",Ingredientes: Aceite de oliva refinado y Aceit...
1,1,4240.0,"Aceite de oliva 0,4º Hacendado","Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva,Botella,1.0,l,4.10,4.10,Ingredientes: Aceite de oliva refinado y Aceit...,x99.,INGREDIENTES | CONSERVACION | Aceite de oliva ...,"Producto: Aceite de oliva 0,4º Hacendado\nID p...","Aceite de oliva 0,4º Hacendado Aceite, especia...","Aceite de oliva 0,4º Hacendado Aceite, especia...",Ingredientes: Aceite de oliva refinado y Aceit...


In [3]:
if not os.getenv('OPENAI_API_KEY'):
    raise RuntimeError(
        'Falta OPENAI_API_KEY. Define la variable en tu entorno o crea '
        f"{ENV_PATH}"
        ' con una linea: OPENAI_API_KEY=tu_clave'
    )

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def _l2_normalize(arr):
    norms = np.linalg.norm(arr, axis=1, keepdims=True) + 1e-12
    return arr / norms

def embed_texts(texts, model=EMBED_MODEL, batch_size=64):
    vectors = []
    for start in range(0, len(texts), batch_size):
        batch = texts[start:start + batch_size]
        resp = client.embeddings.create(model=model, input=batch)
        batch_vecs = [d.embedding for d in resp.data]
        vectors.extend(batch_vecs)
        print(f'Embeddings: {min(start + batch_size, len(texts))}/{len(texts)}')
    arr = np.array(vectors, dtype=np.float32)
    return _l2_normalize(arr)

def ensure_embeddings(chunks_df, force_rebuild=False):
    if EMB_NPY.exists() and not force_rebuild:
        emb = np.load(EMB_NPY)
        if emb.shape[0] == len(chunks_df):
            print('Embeddings cargados desde cache:', EMB_NPY)
            return emb
        print('Cache invalida por numero de filas. Se regenera.')

    emb = embed_texts(chunks_df['text'].tolist())
    np.save(EMB_NPY, emb)
    print('Embeddings guardados en:', EMB_NPY)
    return emb


In [4]:
embeddings = ensure_embeddings(chunks)
embeddings.shape


Embeddings cargados desde cache: /home/wencm/Alimentación/rag_cache/embeddings.npy


(4553, 1536)

In [5]:
import json
import math
import re
import unicodedata
from collections import Counter, defaultdict

SYSTEM_PROMPT = (
    'Eres un asistente experto en el catalogo de Mercadona. '
    'Responde SOLO con la informacion del contexto recuperado. '
    'Si no hay evidencia suficiente, di claramente que no aparece en el dataset. '
    'Si la pregunta es una receta, construye una lista de compra realista para el numero de personas '
    'usando productos presentes en el contexto, y separa siempre: '
    '1) coste de compra real (envases completos), '
    '2) escandallo real del plato (coste consumido). '
    'Si falta precio de algun ingrediente, indicalo explicitamente.'
)

RECIPE_TRIGGERS = (
    'receta', 'rehogado', 'rehogar', 'guiso', 'cocinar', 'ingredientes',
    'personas', 'menu', 'plato', 'preparar'
)

STOPWORDS = {
    'de', 'la', 'el', 'los', 'las', 'un', 'una', 'unos', 'unas', 'para', 'por', 'con', 'sin',
    'que', 'del', 'al', 'en', 'y', 'o', 'a', 'se', 'su', 'sus', 'como', 'dime', 'quiero',
    'hacer', 'preparar', 'receta', 'personas', 'plato', 'menu'
}

# Cantidades base estimadas para 4 personas (unidad compatible con dataset: kg/l/ud)
RECIPE_REQUIREMENTS_BASE_4P = {
    'lentejas': (0.40, 'kg'),
    'arroz': (0.32, 'kg'),
    'marisco': (0.45, 'kg'),
    'caldo de marisco': (1.00, 'l'),
    'caldo': (1.00, 'l'),
    'tomate': (0.20, 'kg'),
    'cebolla': (0.15, 'kg'),
    'ajo': (0.02, 'kg'),
    'pimiento': (0.15, 'kg'),
    'zanahoria': (0.15, 'kg'),
    'aceite de oliva': (0.06, 'l'),
    'sal': (0.01, 'kg'),
    'pimenton': (0.005, 'kg'),
    'laurel': (0.002, 'kg'),
}

def normalize_text(text):
    if text is None:
        return ''
    text = str(text).lower()
    text = unicodedata.normalize('NFD', text)
    return ''.join(ch for ch in text if unicodedata.category(ch) != 'Mn')

def _token_variants(token):
    variants = {token}
    if len(token) > 4 and token.endswith('es'):
        variants.add(token[:-2])
    if len(token) > 3 and token.endswith('s'):
        variants.add(token[:-1])
    return variants

def tokenize(text):
    raw = re.findall(r'[a-z0-9]+', normalize_text(text))
    out = []
    for tok in raw:
        out.extend(_token_variants(tok))
    return out

def _safe_float(v):
    try:
        if v is None:
            return None
        if isinstance(v, str) and not v.strip():
            return None
        return float(v)
    except Exception:
        return None

def _fmt_money(v):
    if v is None or (isinstance(v, float) and math.isnan(v)):
        return '-'
    return f'{v:.2f}'

def _fmt_num(v, digits=3):
    if v is None or (isinstance(v, float) and math.isnan(v)):
        return '-'
    return f'{v:.{digits}f}'

def _to_dim_base(value, unit):
    value = _safe_float(value)
    unit_n = normalize_text(unit)
    if value is None:
        return None, None

    if unit_n == 'kg':
        return 'mass_kg', value
    if unit_n == 'g':
        return 'mass_kg', value / 1000.0
    if unit_n == 'l':
        return 'vol_l', value
    if unit_n == 'ml':
        return 'vol_l', value / 1000.0
    if unit_n in ('ud', 'u', 'unidad', 'unidades'):
        return 'unit_ud', value
    return None, None

def parse_servings(query, default=4):
    qn = normalize_text(query)
    m = re.search(r'(\d+)\s*personas?', qn)
    if m:
        try:
            n = int(m.group(1))
            if n > 0:
                return n
        except Exception:
            pass
    return default

def is_recipe_query(query):
    qn = normalize_text(query)
    return any(trigger in qn for trigger in RECIPE_TRIGGERS)

def _remove_redundant_ingredients(items):
    normalized = [(item, normalize_text(item)) for item in items]
    keep = []

    for i, (item, norm_item) in enumerate(normalized):
        redundant = False
        for j, (_, norm_other) in enumerate(normalized):
            if i == j:
                continue
            if norm_item in norm_other and len(norm_item.split()) < len(norm_other.split()):
                redundant = True
                break
        if not redundant:
            keep.append(item)

    return keep

def _extract_json_list(text):
    text = text.strip()
    if not text:
        return []

    # 1) Try raw JSON first
    try:
        obj = json.loads(text)
        if isinstance(obj, list):
            return obj
        if isinstance(obj, dict) and isinstance(obj.get('ingredients'), list):
            return obj['ingredients']
    except Exception:
        pass

    # 2) Try to capture first JSON list in markdown or prose
    m = re.search(r'\[[\s\S]*\]', text)
    if m:
        try:
            obj = json.loads(m.group(0))
            if isinstance(obj, list):
                return obj
        except Exception:
            pass

    return []

def _clean_ingredient_name(item):
    s = normalize_text(item)
    s = re.sub(r'\([^)]*\)', ' ', s)
    s = re.sub(r'\baprox(?:imadamente)?\b', ' ', s)

    # Remove leading quantity/unit chunks: "400 g de", "2 dientes de", etc.
    s = re.sub(
        r'^\s*\d+(?:[\.,]\d+)?\s*(kg|g|gr|gramos?|ml|l|litros?|unidad(?:es)?|ud|dientes?|cucharad(?:a|as)|taza(?:s)?)?\s*(de\s+)?',
        '',
        s,
    )
    s = re.sub(r'^\s*(unos?|unas?|un|una)\s+', '', s)

    # Remove residual quantities/units inside phrase.
    s = re.sub(r'\b\d+(?:[\.,]\d+)?\b', ' ', s)
    s = re.sub(r'\b(kg|g|gr|gramos?|ml|l|litros?|unidad(?:es)?|ud|dientes?|cucharad(?:a|as)|taza(?:s)?)\b', ' ', s)

    s = re.sub(r'\s+', ' ', s).strip()
    if s.startswith('de '):
        s = s[3:].strip()

    return s

def infer_recipe_ingredients_llm(query, max_items=12):
    prompt = (
        'Extrae ingredientes para la receta solicitada. '
        'Devuelve SOLO un JSON array de strings, sin texto adicional. '
        'No incluyas cantidades ni unidades. '
        'Incluye ingredientes base y condimentos habituales. '
        'No incluyas bebidas, refrescos ni acompanamientos opcionales. '
        f'Consulta: {query}'
    )

    try:
        resp = client.responses.create(
            model=CHAT_MODEL,
            input=prompt,
        )
        raw = (resp.output_text or '').strip()
        arr = _extract_json_list(raw)

        out = []
        seen = set()
        for x in arr:
            if not isinstance(x, str):
                continue
            item = _clean_ingredient_name(x)
            if len(item) < 2:
                continue
            key = normalize_text(item)
            if not key or key in seen:
                continue
            seen.add(key)
            out.append(item)
            if len(out) >= max_items:
                break

        return out
    except Exception as exc:
        print(f'WARN infer_recipe_ingredients_llm fallo: {exc}')
        return []

def infer_recipe_ingredients(query):
    if not is_recipe_query(query):
        return []

    inferred = infer_recipe_ingredients_llm(query)
    if inferred:
        return _remove_redundant_ingredients(inferred)

    # Fallback sin reglas de plato concreto
    tokens = [
        t for t in tokenize(query)
        if len(t) > 2 and t not in STOPWORDS and not t.isdigit()
    ]
    return _remove_redundant_ingredients(tokens[:8])

def _ingredient_requirement_key(ingredient):
    ing_n = normalize_text(ingredient)
    keys = sorted(RECIPE_REQUIREMENTS_BASE_4P.keys(), key=len, reverse=True)
    for key in keys:
        if key in ing_n:
            return key
    return None

def estimate_required_quantity(ingredient, servings=4):
    key = _ingredient_requirement_key(ingredient)
    if key is None:
        return 1.0 * (servings / 4.0), 'ud', 'fallback'

    base_qty, base_unit = RECIPE_REQUIREMENTS_BASE_4P[key]
    factor = servings / 4.0
    return base_qty * factor, base_unit, key

def expand_subqueries(query, recipe_mode='auto'):
    subqueries = [query]
    inferred_ingredients = []

    use_recipe = (recipe_mode is True) or (recipe_mode == 'auto' and is_recipe_query(query))
    if use_recipe:
        inferred_ingredients = infer_recipe_ingredients(query)
        for ingredient in inferred_ingredients:
            subqueries.append(f'{ingredient} mercadona precio')

    return subqueries, inferred_ingredients

# General index (RAG context)
DOC_TOKENS = [tokenize(text) for text in chunks['lexical_text'].fillna('')]
POSTINGS = defaultdict(list)
DOC_FREQ = defaultdict(int)
N_DOCS = len(DOC_TOKENS)

for doc_idx, toks in enumerate(DOC_TOKENS):
    counts = Counter(toks)
    for tok, tf in counts.items():
        POSTINGS[tok].append((doc_idx, tf))
        DOC_FREQ[tok] += 1

# Ingredient-focused indexes
# 1) Name/category index (precision)
ING_NAME_DOC_TOKENS = [tokenize(text) for text in chunks['ingredient_search_text'].fillna('')]
ING_NAME_TOKEN_SETS = [set(toks) for toks in ING_NAME_DOC_TOKENS]
ING_NAME_POSTINGS = defaultdict(list)
ING_NAME_DOC_FREQ = defaultdict(int)

for doc_idx, toks in enumerate(ING_NAME_DOC_TOKENS):
    counts = Counter(toks)
    for tok, tf in counts.items():
        ING_NAME_POSTINGS[tok].append((doc_idx, tf))
        ING_NAME_DOC_FREQ[tok] += 1

# 2) Description/ingredients index (recall)
ING_DESC_DOC_TOKENS = [tokenize(text) for text in chunks['ingredient_desc_text'].fillna('')]
ING_DESC_TOKEN_SETS = [set(toks) for toks in ING_DESC_DOC_TOKENS]
ING_DESC_POSTINGS = defaultdict(list)
ING_DESC_DOC_FREQ = defaultdict(int)

for doc_idx, toks in enumerate(ING_DESC_DOC_TOKENS):
    counts = Counter(toks)
    for tok, tf in counts.items():
        ING_DESC_POSTINGS[tok].append((doc_idx, tf))
        ING_DESC_DOC_FREQ[tok] += 1

def _tfidf_scores_index(q_tokens, postings, doc_freq):
    if not q_tokens:
        return np.zeros(N_DOCS, dtype=np.float32)

    q_counts = Counter(q_tokens)
    scores = np.zeros(N_DOCS, dtype=np.float32)

    for tok, qtf in q_counts.items():
        rows = postings.get(tok)
        if not rows:
            continue

        idf = math.log((N_DOCS + 1) / (doc_freq[tok] + 1)) + 1.0
        q_weight = 1.0 + math.log(qtf)
        for doc_i, tf in rows:
            scores[doc_i] += q_weight * (1.0 + math.log(tf)) * idf

    max_score = float(scores.max())
    if max_score > 0:
        scores /= max_score
    return scores

def lexical_scores(query):
    q_tokens = [t for t in tokenize(query) if len(t) > 2 and t not in STOPWORDS]
    return _tfidf_scores_index(q_tokens, POSTINGS, DOC_FREQ)

def lexical_scores_ingredient(query):
    q_tokens = [t for t in tokenize(query) if len(t) > 2 and t not in STOPWORDS]
    name_scores = _tfidf_scores_index(q_tokens, ING_NAME_POSTINGS, ING_NAME_DOC_FREQ)
    desc_scores = _tfidf_scores_index(q_tokens, ING_DESC_POSTINGS, ING_DESC_DOC_FREQ)

    # Name/category drives precision; ingredients/description improves recall for indirect product names.
    return 0.80 * name_scores + 0.20 * desc_scores

def semantic_scores_batch(queries):
    resp = client.embeddings.create(model=EMBED_MODEL, input=queries)
    all_scores = []
    for item in resp.data:
        q_vec = np.array(item.embedding, dtype=np.float32)
        q_vec = q_vec / (np.linalg.norm(q_vec) + 1e-12)
        sem = embeddings @ q_vec
        all_scores.append((sem + 1.0) / 2.0)
    return all_scores

def _ingredient_token_masks(ingredient):
    tokens = [t for t in tokenize(ingredient) if t not in STOPWORDS and len(t) > 2]
    if not tokens:
        zeros = np.zeros(N_DOCS, dtype=bool)
        return zeros, zeros

    any_mask = np.zeros(N_DOCS, dtype=bool)
    all_mask = np.zeros(N_DOCS, dtype=bool)

    for doc_i, tokset in enumerate(ING_NAME_TOKEN_SETS):
        present = [tok in tokset for tok in tokens]
        if any(present):
            any_mask[doc_i] = True
        if all(present):
            all_mask[doc_i] = True

    return any_mask, all_mask

def retrieve_hybrid(query, top_k=20, alpha=0.65, recipe_mode='auto'):
    subqueries, inferred_ingredients = expand_subqueries(query, recipe_mode=recipe_mode)
    sem_scores_list = semantic_scores_batch(subqueries)

    hybrid_best = np.zeros(N_DOCS, dtype=np.float32)
    semantic_best = np.zeros(N_DOCS, dtype=np.float32)
    lexical_best = np.zeros(N_DOCS, dtype=np.float32)

    for subquery, sem_scores in zip(subqueries, sem_scores_list):
        lex_scores = lexical_scores(subquery)
        hybrid_scores = alpha * sem_scores + (1.0 - alpha) * lex_scores

        hybrid_best = np.maximum(hybrid_best, hybrid_scores)
        semantic_best = np.maximum(semantic_best, sem_scores)
        lexical_best = np.maximum(lexical_best, lex_scores)

    top_idx = np.argsort(-hybrid_best)[:top_k]

    hits = chunks.iloc[top_idx].copy()
    hits['score'] = hybrid_best[top_idx]
    hits['score_semantic'] = semantic_best[top_idx]
    hits['score_lexical'] = lexical_best[top_idx]

    return hits.sort_values('score', ascending=False).reset_index(drop=True), subqueries, inferred_ingredients

def retrieve(query, top_k=8, mode='hybrid', alpha=0.65, recipe_mode='auto'):
    mode = mode.lower()

    if mode == 'semantic':
        sem_scores = semantic_scores_batch([query])[0]
        top_idx = np.argsort(-sem_scores)[:top_k]
        hits = chunks.iloc[top_idx].copy()
        hits['score'] = sem_scores[top_idx]
        return hits.sort_values('score', ascending=False).reset_index(drop=True), [query], []

    if mode == 'hybrid':
        return retrieve_hybrid(query, top_k=top_k, alpha=alpha, recipe_mode=recipe_mode)

    raise ValueError("mode debe ser 'semantic' o 'hybrid'")

def retrieve_products_for_ingredient(ingredient, top_n=10, alpha=0.35, recipe_query=None):
    context_query = normalize_text(recipe_query) if recipe_query else ''
    search_query = f'{ingredient} {context_query} mercadona producto precio'.strip()
    sem_scores = semantic_scores_batch([search_query])[0]
    lex_scores = lexical_scores_ingredient(search_query)

    cols = [
        'row_idx', 'product_id', 'product_name', 'category', 'subcategory',
        'packaging', 'unit_size', 'size_format', 'price_unit', 'price_bulk'
    ]

    # Safety: if there is no lexical evidence at all, return empty instead of noisy semantic matches.
    if float(lex_scores.max()) <= 0.0:
        empty = pd.DataFrame(columns=cols)
        empty['ingredient'] = []
        empty['score_ingredient'] = []
        return empty

    # Ingredient ranking prioritizes lexical precision over semantic recall.
    hybrid_scores = alpha * sem_scores + (1.0 - alpha) * lex_scores

    any_mask, all_mask = _ingredient_token_masks(ingredient)
    if any_mask.any():
        hybrid_scores = np.where(any_mask, hybrid_scores + 0.25, hybrid_scores * 0.10)
    if all_mask.any():
        hybrid_scores = np.where(all_mask, hybrid_scores + 0.15, hybrid_scores)

    candidate_n = min(N_DOCS, max(top_n * 5, top_n))
    top_idx = np.argsort(-hybrid_scores)[:candidate_n]

    out = chunks.iloc[top_idx][cols].copy()
    out['ingredient'] = ingredient
    out['score_ingredient'] = hybrid_scores[top_idx]

    # Deduplicate same product across duplicated category branches.
    out = out.sort_values('score_ingredient', ascending=False).drop_duplicates(subset=['product_id'], keep='first')
    return out.head(top_n).reset_index(drop=True)

def tool_get_products_for_ingredients(ingredients, per_ingredient=10, alpha=0.35, recipe_query=None):
    """
    Tool-like helper: for each ingredient, returns a ranked product shortlist.
    """
    catalog = {}
    for ingredient in ingredients:
        hits = retrieve_products_for_ingredient(ingredient, top_n=per_ingredient, alpha=alpha, recipe_query=recipe_query)
        catalog[ingredient] = hits
    return catalog

def _choose_best_candidate(df_hits, required_unit):
    if df_hits is None or df_hits.empty:
        return None

    req_dim, _ = _to_dim_base(1.0, required_unit)

    best_any = None
    best_dim_match = None

    for _, row in df_hits.iterrows():
        price = _safe_float(row.get('price_unit'))
        unit_size = _safe_float(row.get('unit_size'))
        size_fmt = row.get('size_format')
        dim, pack_base = _to_dim_base(unit_size, size_fmt)

        if price is None:
            continue

        if best_any is None:
            best_any = row

        if req_dim is not None and dim == req_dim and pack_base is not None and pack_base > 0:
            best_dim_match = row
            break

    if best_dim_match is not None:
        return best_dim_match
    if best_any is not None:
        return best_any

    return df_hits.iloc[0]

def _compute_buy_and_escandallo(chosen_row, required_qty, required_unit):
    if chosen_row is None:
        return {
            'units_to_buy': None,
            'purchase_cost_eur': None,
            'escandallo_cost_eur': None,
            'notes': 'Sin producto candidato',
        }

    price_unit = _safe_float(chosen_row.get('price_unit'))
    unit_size = _safe_float(chosen_row.get('unit_size'))
    size_format = chosen_row.get('size_format')

    req_dim, req_base = _to_dim_base(required_qty, required_unit)
    pack_dim, pack_base = _to_dim_base(unit_size, size_format)

    if price_unit is None:
        return {
            'units_to_buy': None,
            'purchase_cost_eur': None,
            'escandallo_cost_eur': None,
            'notes': 'Producto sin price_unit',
        }

    # Regla clave: compra siempre envase completo (redondeo al alza)
    if req_dim is not None and pack_dim == req_dim and pack_base is not None and pack_base > 0 and req_base is not None:
        units_to_buy = max(1, math.ceil(req_base / pack_base))
        purchase_cost = units_to_buy * price_unit
        escandallo_cost = (req_base / pack_base) * price_unit
        return {
            'units_to_buy': units_to_buy,
            'purchase_cost_eur': purchase_cost,
            'escandallo_cost_eur': escandallo_cost,
            'notes': 'OK unidades comparables',
        }

    # Fallback: no se puede convertir unidades -> compra minima 1 unidad.
    return {
        'units_to_buy': 1,
        'purchase_cost_eur': price_unit,
        'escandallo_cost_eur': None,
        'notes': 'Unidad no comparable, compra minima 1 unidad',
    }

def build_recipe_cost_plan(ingredient_catalog, ingredients, query):
    servings = parse_servings(query, default=4)
    rows = []

    for ingredient in ingredients:
        required_qty, required_unit, req_source = estimate_required_quantity(ingredient, servings=servings)
        hits = ingredient_catalog.get(ingredient)
        chosen = _choose_best_candidate(hits, required_unit)

        if chosen is None:
            rows.append({
                'ingredient': ingredient,
                'required_qty': required_qty,
                'required_unit': required_unit,
                'requirement_source': req_source,
                'product_id': None,
                'product_name': None,
                'unit_size': None,
                'size_format': None,
                'price_unit': None,
                'units_to_buy': None,
                'purchase_cost_eur': None,
                'escandallo_cost_eur': None,
                'notes': 'Sin candidatos',
            })
            continue

        cost_info = _compute_buy_and_escandallo(chosen, required_qty, required_unit)
        rows.append({
            'ingredient': ingredient,
            'required_qty': required_qty,
            'required_unit': required_unit,
            'requirement_source': req_source,
            'product_id': chosen.get('product_id'),
            'product_name': chosen.get('product_name'),
            'unit_size': _safe_float(chosen.get('unit_size')),
            'size_format': chosen.get('size_format'),
            'price_unit': _safe_float(chosen.get('price_unit')),
            'units_to_buy': cost_info['units_to_buy'],
            'purchase_cost_eur': cost_info['purchase_cost_eur'],
            'escandallo_cost_eur': cost_info['escandallo_cost_eur'],
            'notes': cost_info['notes'],
        })

    plan_df = pd.DataFrame(rows)

    total_purchase = _safe_float(plan_df['purchase_cost_eur'].dropna().sum()) if not plan_df.empty else 0.0
    total_escandallo = _safe_float(plan_df['escandallo_cost_eur'].dropna().sum()) if not plan_df.empty else 0.0

    missing_purchase = plan_df[plan_df['purchase_cost_eur'].isna()]['ingredient'].tolist() if not plan_df.empty else []
    missing_esc = plan_df[plan_df['escandallo_cost_eur'].isna()]['ingredient'].tolist() if not plan_df.empty else []

    summary = {
        'servings': servings,
        'total_purchase_eur': total_purchase,
        'total_escandallo_eur': total_escandallo,
        'missing_purchase_ingredients': missing_purchase,
        'missing_escandallo_ingredients': missing_esc,
    }

    return plan_df, summary

def format_ingredient_catalog_text(catalog, max_items=6):
    blocks = []
    for ingredient, df_hits in catalog.items():
        lines = [f'Ingrediente: {ingredient}']
        if df_hits.empty:
            lines.append('- Sin candidatos en el dataset')
        else:
            for _, row in df_hits.head(max_items).iterrows():
                lines.append(
                    '- '
                    f"product_id={row['product_id']}; "
                    f"nombre={row['product_name']}; "
                    f"categoria={row['category']}; "
                    f"precio_unit={row['price_unit']}; "
                    f"formato={row['unit_size']} {row['size_format']}; "
                    f"score={row['score_ingredient']:.4f}"
                )
        blocks.append('\n'.join(lines))
    return '\n\n'.join(blocks)

def format_cost_plan_text(plan_df, summary):
    if plan_df is None or plan_df.empty:
        return 'Sin plan de costes.'

    lines = []
    lines.append(f"Personas: {summary['servings']}")
    lines.append(f"TOTAL_COMPRA_EUR (envase completo): {_fmt_money(summary['total_purchase_eur'])}")
    lines.append(f"TOTAL_ESCANDALLO_EUR (consumo real): {_fmt_money(summary['total_escandallo_eur'])}")

    if summary['missing_purchase_ingredients']:
        lines.append('Faltan precios de compra para: ' + ', '.join(summary['missing_purchase_ingredients']))
    if summary['missing_escandallo_ingredients']:
        lines.append('No se pudo calcular escandallo para: ' + ', '.join(summary['missing_escandallo_ingredients']))

    lines.append('')
    lines.append('| ingrediente | product_id | producto | req_qty | pack_size | price_unit | units_to_buy | purchase_cost_eur | escandallo_cost_eur |')
    lines.append('|---|---:|---|---:|---|---:|---:|---:|---:|')

    for _, r in plan_df.iterrows():
        lines.append(
            f"| {r.get('ingredient')} | {r.get('product_id')} | {r.get('product_name')} | "
            f"{_fmt_num(r.get('required_qty'), 3)} {r.get('required_unit')} | "
            f"{_fmt_num(r.get('unit_size'), 3)} {r.get('size_format')} | "
            f"{_fmt_money(r.get('price_unit'))} | {r.get('units_to_buy')} | "
            f"{_fmt_money(r.get('purchase_cost_eur'))} | {_fmt_money(r.get('escandallo_cost_eur'))} |"
        )

    return '\n'.join(lines)


def _fmt_money_es(v):
    n = _safe_float(v)
    if n is None:
        return 'N/D'
    return f"{n:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

def _format_qty_for_recipe_line(ingredient, required_qty, required_unit):
    qty = _safe_float(required_qty)
    unit = normalize_text(required_unit)
    ing = normalize_text(ingredient)

    if qty is None:
        return 'cantidad N/D'

    condiments = ('sal', 'pimienta', 'azafran', 'pimenton', 'laurel')

    if unit == 'kg':
        grams = qty * 1000.0
        if any(c in ing for c in condiments):
            if grams <= 6:
                return '1 cucharadita'
            if grams <= 20:
                return '1 cucharada'
        if grams < 1000:
            return f"{int(round(grams))} g"
        return f"{_fmt_num(qty, 2)} kg"

    if unit == 'l':
        ml = qty * 1000.0
        if 'limon' in ing and ml <= 60:
            return '1 chorro'
        if 'aceite' in ing and ml <= 80:
            return '1 chorro'
        if ml < 1000:
            return f"{int(round(ml))} ml"
        return f"{_fmt_num(qty, 2)} l"

    if unit == 'ud':
        if 'limon' in ing:
            return '1 chorro'
        n = max(1, int(round(qty)))
        return f"{n} ud"

    return f"{_fmt_num(qty, 2)} {required_unit}"

def build_block1_ingredients_mercadona(plan_df):
    if plan_df is None or plan_df.empty:
        return 'No se encontraron ingredientes/productos en Mercadona para esta receta.'

    lines = []
    for _, r in plan_df.iterrows():
        ingredient = r.get('ingredient')
        product_name = r.get('product_name')
        req_qty = r.get('required_qty')
        req_unit = r.get('required_unit')

        if product_name is None or str(product_name) == 'nan':
            lines.append(f"- {ingredient}: sin producto identificado en el dataset")
            continue

        qty_text = _format_qty_for_recipe_line(ingredient, req_qty, req_unit)
        lines.append(f"- {qty_text} {product_name}")

    return '\n'.join(lines)

def tool_get_total_purchase_price(cost_summary):
    total = _safe_float(cost_summary.get('total_purchase_eur')) if isinstance(cost_summary, dict) else None
    if total is None:
        return 'Precio de los productos a comprar para esta receta: N/D'
    return f"Precio de los productos a comprar para esta receta: {_fmt_money_es(total)}€"

def _build_recipe_prompt_context(plan_df, max_items=14):
    if plan_df is None or plan_df.empty:
        return 'No hay productos concretos disponibles.'

    lines = []
    for _, r in plan_df.head(max_items).iterrows():
        product_name = r.get('product_name')
        ingredient = r.get('ingredient')
        req_qty = _fmt_num(r.get('required_qty'), 3)
        req_unit = r.get('required_unit')
        if product_name is None or str(product_name) == 'nan':
            continue
        lines.append(f"- {ingredient}: {product_name} ({req_qty} {req_unit})")

    return '\n'.join(lines) if lines else 'No hay productos concretos disponibles.'

def build_block3_recipe_text(query, plan_df, model=CHAT_MODEL, max_chars=1000):
    context_lines = _build_recipe_prompt_context(plan_df)

    prompt = (
        f"Escribe una receta breve en español para esta solicitud: {query}.\n"
        f"Usa de referencia estos ingredientes/productos:\n{context_lines}\n\n"
        f"Reglas estrictas:\n"
        f"- Máximo {max_chars} caracteres.\n"
        f"- Texto corrido con pasos claros (sin tablas).\n"
        f"- No inventes precios.\n"
        f"- Devuelve solo el texto de la receta."
    )

    raw_resp = None
    text = ''
    try:
        raw_resp = client.responses.create(model=model, input=prompt)
        text = (raw_resp.output_text or '').strip()
    except Exception as exc:
        text = f"No se pudo generar el texto de receta automáticamente ({exc})."

    if len(text) > max_chars:
        text = text[: max_chars - 1].rstrip() + '…'

    return text, raw_resp

def compose_structured_answer(block1, block2, block3):
    return (
        "Ingredientes de Mercadona:\n"
        f"{block1}\n\n"
        f"{block2}\n\n"
        "Receta y pasos:\n"
        f"{block3}"
    )

def _catalog_preview(catalog, n=3):
    preview = {}
    for ing, df_hits in catalog.items():
        preview[ing] = [
            {
                'product_id': r['product_id'],
                'product_name': r['product_name'],
                'price_unit': r['price_unit'],
            }
            for _, r in df_hits.head(n).iterrows()
        ]
    return preview

def ask_agent(
    query,
    top_k=20,
    model=CHAT_MODEL,
    retrieval_mode='hybrid',
    alpha=0.65,
    recipe_mode='auto',
    use_ingredient_tool=True,
    candidates_per_ingredient=10,
):
    hits, subqueries, inferred_ingredients = retrieve(
        query,
        top_k=top_k,
        mode=retrieval_mode,
        alpha=alpha,
        recipe_mode=recipe_mode,
    )

    ingredient_catalog = {}
    ingredient_catalog_text = ''
    cost_plan_df = pd.DataFrame()
    cost_summary = {
        'servings': parse_servings(query, default=4),
        'total_purchase_eur': 0.0,
        'total_escandallo_eur': 0.0,
        'missing_purchase_ingredients': [],
        'missing_escandallo_ingredients': [],
    }
    cost_plan_text = 'Sin plan de costes.'

    if use_ingredient_tool and inferred_ingredients:
        ingredient_catalog = tool_get_products_for_ingredients(
            inferred_ingredients,
            per_ingredient=candidates_per_ingredient,
            alpha=0.35,
            recipe_query=query,
        )
        ingredient_catalog_text = format_ingredient_catalog_text(ingredient_catalog, max_items=6)

        cost_plan_df, cost_summary = build_recipe_cost_plan(
            ingredient_catalog=ingredient_catalog,
            ingredients=inferred_ingredients,
            query=query,
        )
        cost_plan_text = format_cost_plan_text(cost_plan_df, cost_summary)

    block_1 = build_block1_ingredients_mercadona(cost_plan_df)
    block_2 = tool_get_total_purchase_price(cost_summary)
    block_3, raw_resp_block3 = build_block3_recipe_text(query, cost_plan_df, model=model, max_chars=1000)
    structured_answer = compose_structured_answer(block_1, block_2, block_3)

    hit_cols = [
        'product_id', 'product_name', 'category', 'price_unit',
        'unit_size', 'size_format', 'score'
    ]
    for extra in ('score_semantic', 'score_lexical'):
        if extra in hits.columns:
            hit_cols.append(extra)

    return {
        'answer': structured_answer,
        'block_1': block_1,
        'block_2': block_2,
        'block_3': block_3,
        'hits': hits[hit_cols],
        'subqueries': subqueries,
        'inferred_ingredients': inferred_ingredients,
        'ingredient_catalog': ingredient_catalog,
        'ingredient_catalog_preview': _catalog_preview(ingredient_catalog, n=3),
        'cost_plan': cost_plan_df,
        'cost_summary': cost_summary,
        'cost_plan_text': cost_plan_text,
        'ingredient_catalog_text': ingredient_catalog_text,
        'raw_response': raw_resp_block3,
    }



In [6]:
question = 'Dime la receta para paella de marisco para 4 personas'
result = ask_agent(
    question,
    top_k=35,
    retrieval_mode='hybrid',
    alpha=0.65,
    recipe_mode='auto',
    use_ingredient_tool=True,
    candidates_per_ingredient=12,
)
print(result['answer'])
result['cost_plan'].head(20)


Ingredientes de Mercadona:
- 320 g Arroz marinera Hacendado ultracongelado
- 1.00 l Caldo de marisco o fideuá para paella Hacendado con sofrito
- 1 ud Salteado de gambas, espárragos verdes, ajos tiernos, champiñón y cebolla Hacendado ultracongelado
- 1 ud Mejillones en escabeche Hacendado medianos
- 1 ud Calamar troceado limpio Hacendado congelado
- 1 ud Concha fina al natural Hacendado
- 200 g Tomate doble concentrado Hacendado extra
- 150 g Pimiento rojo
- 150 g Cebollas
- 20 g Picada de ajo y perejil
- 1 chorro Aceite de oliva 1º Hacendado
- 1 ud Sazonador para paella con azafrán Hacendado

Precio de los productos a comprar para esta receta: 48,48€

Receta y pasos:
Calienta el aceite de oliva en una paellera y sofríe la cebolla picada, el pimiento rojo en tiras y la picada de ajo y perejil hasta que estén tiernos. Añade el tomate doble concentrado y mezcla. Incorpora los calamares troceados y cocina unos minutos. Agrega el arroz marinera ultracongelado y mezcla bien para que se impr

,ingredient,required_qty,required_unit,requirement_source,product_id,product_name,unit_size,size_format,price_unit,units_to_buy,purchase_cost_eur,escandallo_cost_eur,notes
0,arroz,0.32,kg,arroz,12782.0,Arroz marinera Hacendado ultracongelado,0.350,kg,2.20,1,2.20,2.011429,OK unidades comparables
1,caldo de marisco,1.00,l,caldo de marisco,7021.0,Caldo de marisco o fideuá para paella Hacendad...,1.000,l,2.00,1,2.00,2.000000,OK unidades comparables
2,gambas,1.00,ud,fallback,35900.0,"Salteado de gambas, espárragos verdes, ajos ti...",0.450,kg,2.90,1,2.90,NaN,"Unidad no comparable, compra minima 1 unidad"
3,mejillones,1.00,ud,fallback,18602.0,Mejillones en escabeche Hacendado medianos,0.111,kg,2.80,1,2.80,NaN,"Unidad no comparable, compra minima 1 unidad"
4,calamares,1.00,ud,fallback,24242.0,Calamar troceado limpio Hacendado congelado,NaN,kg,6.00,1,6.00,NaN,"Unidad no comparable, compra minima 1 unidad"
5,almejas,1.00,ud,fallback,18554.0,Concha fina al natural Hacendado,0.090,kg,1.90,1,1.90,NaN,"Unidad no comparable, compra minima 1 unidad"
6,tomate,0.20,kg,tomate,16074.0,Tomate doble concentrado Hacendado extra,0.170,kg,1.05,2,2.10,1.235294,OK unidades comparables
7,pimiento rojo,0.15,kg,pimiento,69310.0,Pimiento rojo,0.270,kg,0.78,1,0.78,0.433333,OK unidades comparables
8,cebolla,0.15,kg,cebolla,69079.0,Cebollas,2.000,kg,3.90,1,3.90,0.292500,OK unidades comparables
9,ajo,0.02,kg,ajo,40475.0,Picada de ajo y perejil,0.100,kg,1.50,1,1.50,0.300000,OK unidades comparables


## Notas
- Si cambias el Excel, vuelve a ejecutar la celda de `chunks` y luego `ensure_embeddings(..., force_rebuild=True)`.
- Puedes bajar coste usando menos `top_k` o cambiando a un modelo chat mas pequeno.
- Puedes guardar respuestas historicas en CSV si quieres trazabilidad de preguntas/respuestas.
